In [1]:
import os
import argparse
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ibm import WatsonxLLM
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods
from langchain_community.tools.tavily_search import TavilySearchResults

/Users/coredump/anaconda3/envs/langraph/lib/python3.12/site-packages/langchain_community/tools/tavily_search/__init__.py:3: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.tools.tavily_search.tool import (
/Users/coredump/anaconda3/envs/langraph/lib/python3.12/site-packages/pydantic/_internal/_generate_schema.py:775: UserWarning: Mixing V1 models and V2 models (or constructs, like `TypeAdapter`) is not supported. Please upgrade `TavilySearchAPIWrapper` to V2.
  warn(


In [2]:
from dotenv import load_dotenv

In [3]:
# Fetch Watsonx credentials
WATSONX_URL = os.getenv("WATSONX_URL")
WATSONX_API_KEY = os.getenv("WATSONX_API_KEY")
WATSONX_PROJECT_ID = os.getenv("WATSONX_PROJECT_ID")

In [4]:
# Debugging: Print error if any are missing
if not WATSONX_URL or not WATSONX_API_KEY or not WATSONX_PROJECT_ID:
    raise ValueError("Missing Watsonx credentials. Please set WATSONX_URL, WATSONX_API_KEY, and WATSONX_PROJECT_ID.")


In [5]:
# Initialize Tavily search with API key
tavily_search = TavilySearchResults(max_results=5, tavily_api_key=os.getenv("TAVILY_API_KEY"))

In [6]:
# Define the Internet Researcher Agent
class InternetResearcherAgent:
    """Agent that researches current market trends based on shelf description."""
    
    def __init__(self):
        self.llm = WatsonxLLM(
            model_id="mistralai/mistral-large",
            url=WATSONX_URL,
            apikey=WATSONX_API_KEY,
            project_id=WATSONX_PROJECT_ID,
            params={
                GenParams.TEMPERATURE: 0.5,
                GenParams.MAX_NEW_TOKENS: 1000
            }
        )
    
    def __call__(self, state):
        print("DEBUG: InternetResearcherAgent received state:", state)
        search_queries = [
            f"current retail trends for {state['shelf_description']}",
            f"popular products in {state['shelf_description']} category 2025"
        ]
        research_results = []
        for query in search_queries:
            try:
                results = tavily_search.run(query)
                research_results.append({"query": query, "results": results})
            except Exception as e:
                research_results.append({"query": query, "error": str(e)})
        
        human_message = HumanMessage(
            content=f"""Shelf description: {state['shelf_description']}\n\nSearch results: {research_results}\n\nProvide market trends and recommendations."""
        )
        response = self.llm.invoke([human_message])
        print("DEBUG: InternetResearcherAgent LLM response:", response)
        
        # Return updated state
        new_state = state.copy()
        new_state["market_trends"] = response
        new_state["recommendations"] = response
        return new_state

In [7]:
# Define the Market Analyst Agent
class MarketAnalystAgent:
    """Agent that creates an action plan based on recommendations."""
    
    def __init__(self):
        self.llm = WatsonxLLM(
            model_id="mistralai/mistral-large",
            url=WATSONX_URL,
            apikey=WATSONX_API_KEY,
            project_id=WATSONX_PROJECT_ID,
            params={
                GenParams.TEMPERATURE: 0.3,
                GenParams.MAX_NEW_TOKENS: 1500
            }
        )
    
    def __call__(self, state):
        print("DEBUG: MarketAnalystAgent received state:", state)
        human_message = HumanMessage(
            content=f"""Shelf description: {state['shelf_description']}\nRecommendations: {state['recommendations']}\n\nCreate an action plan."""
        )
        response = self.llm.invoke([human_message])
        print("DEBUG: MarketAnalystAgent LLM response:", response)
        
        # Return updated state
        new_state = state.copy()
        new_state["action_plan"] = response
        new_state["final_output"] = response
        return new_state

In [8]:
# Define workflow graph
def create_shelf_analysis_graph():
    workflow = StateGraph(dict)
    
    # Initialize agents
    researcher = InternetResearcherAgent()
    analyst = MarketAnalystAgent()
    
    # Add nodes
    workflow.add_node("researcher", researcher)
    workflow.add_node("analyst", analyst)
    
    # Add edges
    workflow.add_edge("researcher", "analyst")
    workflow.add_edge("analyst", END)
    
    # Set entry point
    workflow.set_entry_point("researcher")
    
    return workflow.compile()

In [9]:
# Main function to run the analysis
def analyze_shelf(shelf_description):
    graph = create_shelf_analysis_graph()
    initial_state = {
        "shelf_description": shelf_description,
        "market_trends": None,
        "recommendations": None,
        "action_plan": None,
        "final_output": None
    }
    
    # Use the run method instead of stream for simplicity
    try:
        final_state = graph.invoke(initial_state)
        if "final_output" in final_state and final_state["final_output"]:
            return final_state["final_output"]
        else:
            return "Error: Final output not found in state"
    except Exception as e:
        return f"Error: {str(e)}"

In [11]:
if __name__ == "__main__":
    # Define the user prompt inside the script
    shelf_description = (
        "The image shows a display of Adidas shoes on shelves. The shelves are made of black metal bars "
        "attached to a brick wall background. The shoes are 19 pairs of different colored Adidas sneakers "
        "stacked on five metal shelves. All of the shoes are in color and are well placed on the shelf, but each "
        "of them has a label attached that names a different photographer. For example, the image ID, the website, "
        "and some identification numbers. There is also a copyright claim in the image, due to the 'alamy' and 'a' "
        "texts scattered throughout the image. Also, there is a black bar at the bottom of the image, where the name "
        "of the picture style and web address of the provider, 'alamy,' are located. The quality of the image is "
        "impressive as each element in the picture is depicted vividly, and has well-defined detail. Overall, the "
        "image displays how Adidas shoes can be displayed and make it appealing to people who might be interested in "
        "them from all demographics."
    )

    print(f"Analyzing shelf: {shelf_description}")
    result = analyze_shelf(shelf_description)
    print("\nFinal Recommendations and Action Plan:")
    print(result)

Analyzing shelf: The image shows a display of Adidas shoes on shelves. The shelves are made of black metal bars attached to a brick wall background. The shoes are 19 pairs of different colored Adidas sneakers stacked on five metal shelves. All of the shoes are in color and are well placed on the shelf, but each of them has a label attached that names a different photographer. For example, the image ID, the website, and some identification numbers. There is also a copyright claim in the image, due to the 'alamy' and 'a' texts scattered throughout the image. Also, there is a black bar at the bottom of the image, where the name of the picture style and web address of the provider, 'alamy,' are located. The quality of the image is impressive as each element in the picture is depicted vividly, and has well-defined detail. Overall, the image displays how Adidas shoes can be displayed and make it appealing to people who might be interested in them from all demographics.
DEBUG: InternetResearc